<a href="https://colab.research.google.com/github/avgCoderr/Image-Captioning/blob/main/Image_Captioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import numpy as np  
import pandas as pd  
import os
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.layers import Flatten, Dense, LSTM, Dropout, Embedding, Activation
from keras.layers import concatenate, BatchNormalization, Input
from keras.layers.merge import add
from tensorflow.keras.utils import to_categorical, plot_model
from keras.applications.inception_v3 import InceptionV3, preprocess_input
import matplotlib.pyplot as plt
import cv2

In [9]:
# descriptions model

def load_description(text):
    mapping = dict()
    for line in text.split("\n"):
        token = line.split("\t")
        if len(line) < 2:                  # remove short descriptions
            continue
        img_id = token[0].split('.')[0]    # name of the image
        img_des = token[1]                 # description of the image
        if img_id not in mapping:
            mapping[img_id] = list()
        mapping[img_id].append(img_des)
    return mapping

In [10]:
token_path = '/content/gdrive/MyDrive/Colab Notebooks/datasets/img captioning/Flickr8k.token.txt'
text = open(token_path, 'r', encoding = 'utf-8').read()

In [17]:
# forming descriptions of all captions and each set of captions associated with their parent key, 
# which is the name of image

descriptions = load_description(text)
descriptions['1000268201_693b08cb0e']

['A child in a pink dress is climbing up a set of stairs in an entry way .',
 'A girl going into a wooden building .',
 'A little girl climbing into a wooden playhouse .',
 'A little girl climbing the stairs to her playhouse .',
 'A little girl in a pink dress going into a wooden cabin .']

In [15]:
# refining descriptions by removing punctuations, hanging letters and lowercasing 
import string

def clean_description(desc):
    for key, des_list in desc.items():
        for i in range(len(des_list)):
            caption = des_list[i]
            caption = [ch for ch in caption if ch not in string.punctuation]   
            caption = ''.join(caption)
            caption = caption.split(' ')
            caption = [word.lower() for word in caption if len(word)>1 and word.isalpha()]
            caption = ' '.join(caption)
            des_list[i] = caption

In [18]:
clean_description(descriptions)
descriptions['1000268201_693b08cb0e']

#changes can be compared before and after

['child in pink dress is climbing up set of stairs in an entry way',
 'girl going into wooden building',
 'little girl climbing into wooden playhouse',
 'little girl climbing the stairs to her playhouse',
 'little girl in pink dress going into wooden cabin']

In [21]:
# defining a vocabulory of all the words (unique)
# word preprocessing for NLP

def to_vocab(desc):
    words = set()
    for key in desc.keys():
        for line in desc[key]:
            words.update(line.split())
    return words
vocab = to_vocab(descriptions)

In [26]:
# all unique words

vocab

{'treeless',
 'whom',
 'boats',
 'bareback',
 'dupar',
 'reflection',
 'muzzled',
 'withered',
 'taken',
 'wicket',
 'mushing',
 'swingset',
 'dodging',
 'tongues',
 'white',
 'poised',
 'own',
 'pearls',
 'crane',
 'story',
 'surrounding',
 'bee',
 'descend',
 'swampy',
 'dense',
 'spot',
 'darked',
 'soars',
 'trams',
 'sell',
 'note',
 'regains',
 'acoustic',
 'gathers',
 'domes',
 'fribee',
 'cots',
 'windboarder',
 'thousand',
 'change',
 'knelt',
 'bass',
 'umbrella',
 'born',
 'print',
 'outfield',
 'footprints',
 'askance',
 'spotted',
 'aveda',
 'idea',
 'done',
 'hate',
 'weight',
 'thw',
 'aerial',
 'grazes',
 'swimmies',
 'truck',
 'resembling',
 'nurses',
 'establishment',
 'tasting',
 'secured',
 'specialized',
 'hair',
 'curve',
 'blackyellow',
 'mitt',
 'get',
 'garb',
 'du',
 'startled',
 'decorative',
 'pour',
 'binocular',
 'peace',
 'races',
 'everyone',
 'listens',
 'apart',
 'decorations',
 'wave',
 'coutryside',
 'leashes',
 'land',
 'tax',
 'bags',
 'para',
 'op

In [27]:
#image loading baad mei abhi neend aa rahi hai